# Task 

Analyze data, given


Read more [here](./new.csv) # It works!

In [35]:
import xlrd
import csv

In [36]:
def csv_from_excel(xlsx_name, csv_name):
    wb = xlrd.open_workbook(xlsx_name)
    sh = wb.sheet_by_index(0)
    your_csv_file = open(csv_name, 'w')
    wr = csv.writer(your_csv_file, quoting=csv.QUOTE_ALL)

    for rownum in range(sh.nrows):
        wr.writerow(sh.row_values(rownum))

    your_csv_file.close()

In [37]:
csv_from_excel('Послойное описание с фотографиями.xlsx', 'new.csv')

In [38]:
import pandas as pd

In [39]:
df = pd.read_csv("new.csv")

In [23]:
df.columns = list(df.iloc[0])

In [52]:
df["Unnamed: 2"][0] = "Интервал отбора керна, м (Кровля)"

In [53]:
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37
0,Месторождение,Скважина,"Интервал отбора керна, м (Кровля)",NaN,"Интервал отбора керна, м (после привязки)",NaN,Фактический вынос,NaN,Стратиграфическая единица,Слой,...,Пустотное пространство,Трещиноватость (морфологические характеристики...,"Нефтенасыщение/ битуминозность (характер, инте...",Коллектор (тип)/не коллектор,Целостность керна,Границы слоя (контакт с нижележащим слоем),Примечание,"Профильные методы,\nсреднее значение по слою",NaN,NaN
1,NaN,NaN,Кровля,Подошва,Кровля,Подошва,м,%,NaN,Кровля,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Общая радио активность, API","Плотность, г/см3","Проницаемость, мД"
2,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0
3,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.9883333333333334,НП\2-3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,65.92171964285714,2.4122772142857145,2.0791481481481484
4,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.9883333333333334,НП\2-3,1.45,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.73452195238094,2.4861752380952384,0.4906315789473685
5,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.9883333333333334,НП\2-3,3.54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,66.10536595652174,2.441882608695652,22.843947368421055
6,1.0,2.0,1957.1,1963.1,1958.3,1964.3,5.93,0.9883333333333334,НП\2-3,5.85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.195236,2.443174,38.45
7,1.0,2.0,1963.1,1978.1,1964.3,1979.3,15.0,1.0,НП\2-3,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.73426225,2.458811666666667,22.261125
8,1.0,2.0,1963.1,1978.1,1964.3,1979.3,15.0,1.0,НП\2-3,1.26,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.9611245,2.5588245,3.415
9,1.0,2.0,1963.1,1978.1,1964.3,1979.3,15.0,1.0,НП\2-3,1.83,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.998703,2.4342953333333335,13.753
